In [3]:
import os
import json
import pandas as pd

# 🗂️ Thư mục chứa toàn bộ file JSON crawl được
path = "Collected-Data"
files = [f for f in os.listdir(path) if f.endswith(".json")]

records = []

for f in files:
    file_path = os.path.join(path, f)
    try:
        with open(file_path, "r", encoding="utf-8") as fp:
            data = json.load(fp)
    except Exception as e:
        print(f"[!] Không đọc được file {f}: {e}")
        continue

    shop = data.get("shop_name", "Unknown Shop")
    menu = data.get("menu", [])
    feedbacks = data.get("feedback", [])

    # Nếu JSON có danh sách món (menu)
    if isinstance(menu, list) and len(menu) > 0:
        for dish in menu:
            if isinstance(dish, dict):
                dish_name = dish.get("dish_name") or dish.get("name") or "Unknown Dish"
            elif isinstance(dish, str):
                dish_name = dish.strip()
            else:
                dish_name = "Unknown Dish"

            item = f"{shop} - {dish_name}"

            for fb in feedbacks:
                user = fb.get("username", "Anonymous")
                rating = fb.get("review_point", None)
                try:
                    rating = float(rating)
                except (ValueError, TypeError):
                    continue
                records.append([user, item, rating])

    # Nếu không có menu (feedback cho cả quán)
    else:
        for fb in feedbacks:
            user = fb.get("username", "Anonymous")
            rating = fb.get("review_point", None)
            try:
                rating = float(rating)
            except (ValueError, TypeError):
                continue
            item = shop
            records.append([user, item, rating])

# ===============================================
# 🔧 Tạo DataFrame và ma trận user–item đầy đủ
# ===============================================
df = pd.DataFrame(records, columns=["user", "item", "rating"])

if df.empty:
    raise ValueError("❌ Không có dữ liệu hợp lệ trong thư mục Collected-Data!")

# Gộp trùng nếu 1 user đánh giá 1 item nhiều lần
df = df.groupby(["user", "item"], as_index=False)["rating"].mean()

# Tạo ma trận user–item
matrix = df.pivot_table(index="user", columns="item", values="rating", fill_value=0)

# ===============================================
# 💾 Xuất file theo format: user001,0.0,0.0,10.0,...
# ===============================================
output_path = "matrix_full.csv"

# reset index để user thành cột đầu tiên
matrix.reset_index(inplace=True)

# format rating về dạng float có 1 chữ số sau dấu phẩy
matrix = matrix.round(1)

# lưu ra file CSV, không có header item, chỉ dữ liệu
with open(output_path, "w", encoding="utf-8-sig") as f:
    for _, row in matrix.iterrows():
        line = ",".join(map(str, row.values))
        f.write(line + "\n")

# ===============================================
# 📊 Thống kê
# ===============================================
zero_count = (matrix.iloc[:, 1:] == 0).sum().sum()
total = matrix.iloc[:, 1:].shape[0] * matrix.iloc[:, 1:].shape[1]
sparsity = zero_count / total * 100

print("\n✅ Ma trận user–item đầy đủ đã tạo thành công!")
print(f"➡️ Số user: {matrix.shape[0]}")
print(f"➡️ Số item: {matrix.shape[1] - 1}")
print(f"➡️ Tổng số ô: {total}")
print(f"➡️ Số ô = 0: {zero_count} ({sparsity:.2f}% sparse)")
print(f"📁 Đã lưu tại: {output_path}\n")

print(matrix.head(5))



✅ Ma trận user–item đầy đủ đã tạo thành công!
➡️ Số user: 1850
➡️ Số item: 179
➡️ Tổng số ô: 331150
➡️ Số ô = 0: 328850 (99.31% sparse)
📁 Đã lưu tại: matrix_full.csv

item                                   user  \
0     - Đào Tấn Nhà Hàng Nhật Bản Taki Taki   
1                               01672020678   
2                                0932605126   
3                                   1 Volam   
4                                      1626   

item  3 Râu - Gà Rán, Pizza & Trà Sữa - Cây Trâm  \
0                                            0.0   
1                                            0.0   
2                                            0.0   
3                                            0.0   
4                                            0.0   

item  A Tùng - Bánh Mì Bò Nướng Bơ Cambodia - Lê Hồng Phong  An Nam Cháo  \
0                                                   0.0             10.0   
1                                                   0.0              0.0   
2       